In [150]:
import pandas as pd
import pycountry_convert as pc
from pycountry import countries

## Model
suicide ~ population \
suicide ~ population + gdp_per_capita \
group by country (or year) (or age) suicide ~ population + gdp_per_capita

In [151]:
with_hdi = False  # Do you want to create a new df with hdi included?

suicide_df = pd.read_csv('data/suicide.csv')
suicide_df = suicide_df.rename(columns={'suicides_no':'suicide', ' gdp_for_year ($) ':'total_gdp', 'gdp_per_capita ($)':'gdp_per_capita', 'HDI for year':'hdi'})
if with_hdi: 
    suicide_df = suicide_df.drop(columns=['suicides/100k pop', 'country-year', 'gdp_per_capita', 'generation'])
else: 
    suicide_df = suicide_df.drop(columns=['suicides/100k pop', 'hdi', 'country-year', 'gdp_per_capita', 'generation'])
suicide_df['total_gdp'] = suicide_df['total_gdp'].apply(lambda x: int(x.replace(',', '')))
suicide_df      

,country,year,sex,age,suicide,population,total_gdp
0,Albania,1987,male,15-24 years,21,312900,2156624900
1,Albania,1987,male,35-54 years,16,308000,2156624900
2,Albania,1987,female,15-24 years,14,289700,2156624900
3,Albania,1987,male,75+ years,1,21800,2156624900
4,Albania,1987,male,25-34 years,9,274300,2156624900
...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,63067077179
27816,Uzbekistan,2014,female,75+ years,9,348465,63067077179
27817,Uzbekistan,2014,male,5-14 years,60,2762158,63067077179
27818,Uzbekistan,2014,female,5-14 years,44,2631600,63067077179


## Get the rows belong to European countries

In [152]:
# Get the country names since some countries are not recognized by pycountry
countries_name_list = [country.name for country in list(countries)]

# Function to extract continent code
def get_continent(country):
    country_code = pc.country_name_to_country_alpha2(country)
    return pc.country_alpha2_to_continent_code(country_code)

suicide_df = suicide_df[suicide_df['country'].isin(countries_name_list)]   # Filter out invalid countries
suicide_df['continent'] = suicide_df['country'].apply(get_continent)       # Extract continent code
suicide_df = suicide_df[suicide_df['continent'] == 'EU']                   # Get the countries in EU
suicide_df.head(5)

C:\Users\dng09\AppData\Local\Temp\ipykernel_10044\1392511374.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicide_df['continent'] = suicide_df['country'].apply(get_continent)       # Extract continent code


,country,year,sex,age,suicide,population,total_gdp,continent
0,Albania,1987,male,15-24 years,21,312900,2156624900,EU
1,Albania,1987,male,35-54 years,16,308000,2156624900,EU
2,Albania,1987,female,15-24 years,14,289700,2156624900,EU
3,Albania,1987,male,75+ years,1,21800,2156624900,EU
4,Albania,1987,male,25-34 years,9,274300,2156624900,EU


## Group data by year and sex

-   The dataframe is grouped by sex, country, and year. The aggregate function used in this part is sum
-   Used unstack and tranpose to get a dataframe where the columns are the years, indexing by sex and country
-   Remove `nan` values. A column must have data on at least 20/32 years to be staying here
-   Using interpolation and bfill to fill in the missing values. This is why columns that have too much `nan` value needed to be removed as they might affect the accuracy of the interpolation

In [156]:
if with_hdi:
    suicide_df_grouped = suicide_df.groupby(['sex', 'country', 'year']).aggregate({'suicide':'sum', 'population':'sum', 'total_gdp':'sum', 'hdi':'mean'})
    rindices = ['suicide', 'population', 'total_gdp', 'hdi']
    suicide_df_grouped = suicide_df_grouped.unstack().T
    suicide_df_grouped.dropna(thresh=60, axis=1, inplace=True)
else:
    suicide_df_grouped = suicide_df.groupby(['sex', 'country', 'year']).aggregate('sum')
    rindices = ['suicide', 'population', 'total_gdp']
    suicide_df_grouped = suicide_df_grouped.unstack().T
    suicide_df_grouped.dropna(thresh=62, axis=1, inplace=True) 

rindex = 'population'
cindex = 25
test = suicide_df_grouped.copy()
test[suicide_df_grouped.columns[cindex]][[rindex]] = suicide_df_grouped[suicide_df_grouped.columns[cindex]][[rindex]].interpolate(method='linear').bfill()
test[suicide_df_grouped.columns[cindex]][[rindex]]



for cindex in range(suicide_df_grouped.shape[1]):
    for rindex in rindices:
        suicide_df_grouped[suicide_df_grouped.columns[cindex]][[rindex]] = suicide_df_grouped[suicide_df_grouped.columns[cindex]][[rindex]].interpolate(method='linear').bfill()

suicide_df_grouped.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 96 entries, ('suicide', 1985) to ('total_gdp', 2016)
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   (female, Albania)             96 non-null     float64
 1   (female, Austria)             96 non-null     float64
 2   (female, Belarus)             96 non-null     float64
 3   (female, Belgium)             96 non-null     float64
 4   (female, Bulgaria)            96 non-null     float64
 5   (female, Croatia)             96 non-null     float64
 6   (female, Denmark)             96 non-null     float64
 7   (female, Estonia)             96 non-null     float64
 8   (female, Finland)             96 non-null     float64
 9   (female, France)              96 non-null     float64
 10  (female, Germany)             96 non-null     float64
 11  (female, Greece)              96 non-null     float64
 12  (female, Hungary)             96 

## Generate new columns

-   The dataframe is tranposed and stacked so that it have a better form
-   Columns `suicides_per_100k` and `gdp_per_capita` are generated using these data
-   The dataframe is exported to a csv file for future processes

In [157]:
output_df = suicide_df_grouped.T.stack()
output_df['suicides_per_100k'] = (output_df['suicide']/output_df['population']) * 100000
output_df['gdp_per_capita'] = output_df['total_gdp']/output_df['population']
output_df.to_csv('data/suicide_cleaned.csv')

suicide  population     total_gdp  \
sex    country        year                                      
female Albania        1985     25.0   1316900.0  1.293975e+10   
                      1986     25.0   1316900.0  1.293975e+10   
                      1987     25.0   1316900.0  1.293975e+10   
                      1988     22.0   1343600.0  1.275600e+10   
                      1989     15.0   1363300.0  1.401075e+10   
...                             ...         ...           ...   
male   United Kingdom 2012   3512.0  29269825.0  1.597251e+13   
                      2013   3831.0  29477380.0  1.643891e+13   
                      2014   3717.0  29731382.0  1.813697e+13   
                      2015   3732.0  30010716.0  1.731342e+13   
                      2016   3732.0  30010716.0  1.731342e+13   

                            suicides_per_100k  gdp_per_capita  
sex    country        year                                     
female Albania        1985           1.898398     9825.916470  
                      1986           1.898398     9825.916470  
                      1987           1.898398     9825.916470  
                      1988           1.637392     9493.896993  
                      1989           1.100271    10277.084961  
...                                       ...             ...  
male   United Kingdom 2012          11.998705   545698.889932  
                      2013          12.996406   557678.873956  
                      2014          12.501942   610027.703767  
                      2015          12.435558   576907.990298  
                      2016          12.435558   576907.990298  

[2112 rows x 5 columns]